# Filtering and aggregation
- **1. Keep only food products:** Using the item list with the binary variable ('Is food'). -> Could be moved to preprocessing. 
- **2. Aggregate products:** Use FoodEx classification to aggregate food products into Food categories to simplify the analysis. 
- **3. Explore data availability:** We want to see how many countries are trading per product and year (nodes network) and how many transactions (edges network) exist among them. 
- **4. Temporal aggregation** See if data availability is better after aggregating the data into 2-year intervals (36years/2= 18intervals). 
- **5. Filter non-relevant products:** Remove products with 0 transactions for more than half of the intervals. They don't have enough data.


In [1]:
import pandas as pd
import numpy as np
import time
import networkx as nx
import geopandas as gpd #pip installed
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt 
import ast
import seaborn as sns
import openpyxl

In [2]:
# FUNCTION DEFINITION
# working
def Food_group_sum (data, f_group):
    filt_data= data.loc[data.Food_group == f_group,:]
    data_grouped= filt_data.groupby(['Food_group','unit','origin_country','destin_country','year'])
    filt_data.loc[:,'value']= data_grouped.value.transform('sum')

    filt_data.loc[:,'item']= f_group
    return(filt_data)

def FoodEx_aggregation (data):
    food_groups = data.Food_group.unique() 

    # Iterate per food group
    data_fg= Food_group_sum(data,food_groups[0])
    
    for f in food_groups[1:]: 
        data_fg= pd.concat([data_fg, Food_group_sum(data,f)], ignore_index=True)
        print(f)

    data_out = data_fg.drop_duplicates()
    return data_out.drop(columns='item')

def Links_per_year(data_g,year_estimation,key):
    trades_year= data_g.groupby('year').apply(lambda group: group.shape[0]) #num trades per year.
    year_estimation.loc[key, trades_year.index]= trades_year    

def Countries_trading_per_year(data_g,year_estimation,key):
    trades_year= data_g.groupby('year').apply(lambda group: len(set(group.origin_country_ISO.unique()).
                                              union(group.destin_country_ISO.unique()))) #num trades per year.
    year_estimation.loc[key, trades_year.index]= trades_year    

def Year_group_sum (data, year_split):
    """
    Aggregate yearly data based on a specified list of year ranges.

    Parameters:
    - data (pd.DataFrame): Input DataFrame containing columns 'year', 'item', 'unit', 'origin_country', 'destin_country', and 'value'.
    - year_split (list): List of years to filter and aggregate.

    Returns:
    pd.DataFrame: DataFrame with aggregated values for the specified year ranges.
    
    Example:
    >>> input_data = pd.DataFrame({'year': [2010, 2011, 2011, 2012, 2013],
                                   'item': ['A', 'B', 'A', 'B', 'A'],
                                   'unit': ['KG', 'KG', 'L', 'L', 'KG'],
                                   'origin_country': ['US', 'CA', 'US', 'CA', 'US'],
                                   'destin_country': ['CA', 'US', 'CA', 'US', 'CA'],
                                   'value': [10, 20, 30, 40, 50]})
    >>> years_to_aggregate = [2011, 2012, 2013]
    >>> result_df = Year_aggregation(input_data, years_to_aggregate)
    """
    filt_data= data.loc[data.year.isin(year_split),:]
    year_label = str(min(year_split))+'-'+str(max(year_split))

    data_grouped= filt_data.groupby(['Food_group','unit','origin_country','destin_country'])
    filt_data.loc[:,'value']= data_grouped['value'].transform('sum')
    filt_data.loc[:,'year']= year_label
    print(year_split)
    return(filt_data)

def Year_aggregation(data, Num_parts):
    # 36 timepoints 
    year_range = np.arange(data.year.min(),data.year.max()+1)

    split_years = np.array_split(year_range, Num_parts)

    # Year aggregation
    out_data = Year_group_sum(data,split_years[0])

    for s in split_years[1:]: 
        out_data=pd.concat([out_data, Year_group_sum(data,s)], ignore_index=True)

    data_agr = out_data.drop_duplicates()
    return data_agr
    
def Clean_item_names(data, item_list,foodex_levels):
    """Clean `item` names from 'data' to match 'item_list'.

    The function cleans special characters in item names and matches the items between 'data' and 'item_list'
    to guarantee the consistancy between both dataframes. 

    Parameters:
    - data (pd.DataFrame): Input DataFrame containing an 'item' column to be cleaned and filtered.
    - item_list (pd.DataFrame): DataFrame containing the metadata file for each item.

    Returns:
    Tuple: A tuple containing two DataFrames:
        1. Cleaned and filtered DataFrame 'data' with item names matched to the 'item_list'.
        2. Original 'item_list' DataFrame with cleaned item names.

    Example:
    >>> cleaned_data, cleaned_item_list = Clean_item_names(input_data, valid_items)
    """
    
    # Load item codes
    replacements = {'�': 'é', ';': ',', '"': ''}

    item_list['item']=item_list.item.replace(replacements,regex=True)

    data['item']=data.item.replace(replacements,regex=True)

    # Match items
    # print('Items in data, missing in item list: ',len(set(data.item.unique()).difference(set(item_list.item.unique())))) # Check only to confirm that we are not missing any product in item_list
    # print('Items in item_list, not in data: ',set(item_list.item.unique()).difference(set(data.item.unique()))) # Check only to confirm that we are not missing any product in item_list
    
    # Add Foodex labels to item_list 
    item_list= pd.merge(item_list, foodex_levels,left_on='L1_foodex',right_on='Code',how='left').drop(columns='Code').rename(columns={'IndentedTree':'Food_group'})

    # Merge itemlist & data:
    data= pd.merge(data, item_list.loc[:,['item','Is_food','L1_foodex','Food_group']],on='item',how='left',copy=False)
    print(data)
    return data, item_list

def Check_data_available(data):
    # Define matrix
    data_g = data.groupby(['Food_group','unit'])

    dict_keys=data_g.groups.keys()
    
    if isinstance(data.year.iloc[0], str):
        year_range =data.year.unique()
    else: 
        year_range = np.arange(data.year.min(),data.year.max()+1)
    
    year_estimation= pd.DataFrame(np.zeros([len(dict_keys),len(year_range)]),columns=year_range,index=dict_keys)

    # Define matrix
    year_estimation= pd.DataFrame(np.zeros([len(dict_keys),len(year_range)]),columns=year_range,index=dict_keys)
    year_estimation_links=year_estimation.copy()

    # Fill matrices
    for i in list(dict_keys):
        Countries_trading_per_year(data_g.get_group(i),year_estimation, i)

        Links_per_year(data_g.get_group(i),year_estimation_links, i)
    return year_estimation, year_estimation_links, year_range

def Plot_data_available(year_estimation,year_estimation_links,year_range,flag_save): 
    fig, ax = plt.subplots(1,2,figsize=(10,5))
    flat_links= year_estimation_links.to_numpy().flatten()
    ax[0].hist(flat_links,bins = 30, density=True)
    ax[0].set_ylabel('Probability')
    ax[0].set_xlabel('Average. num of transactions')
    ax[0].set_title('Distribution of the av. num of transactions')

    # Plot data availability for aggregates:
    #year_est_bin= 1*(year_estimation>0)
    '''
    mean_size = 100*((year_est_bin).sum())/year_estimation.shape[0]
    
    ax[0].plot(year_range,mean_size)
    ax[0].set_ylim(0,100)
    ax[0].set_ylabel('Percentage of food_groups with data(%)')
    ax[0].title.set_text('Food_grups with some trade data')
    '''
    # Mean number of links per year
    mean_links = year_estimation.median(axis=0)

    inf_perc = np.quantile(year_estimation, 0.025,axis=0)
    sup_perc = np.quantile(year_estimation, 0.975,axis=0)
            
    std_links = year_estimation.std()

    ax[1].plot(year_range, mean_links, color='blue', alpha=0.2, label='Standard Deviation')
    ax[1].fill_between(year_range, inf_perc, sup_perc, color='blue', alpha=0.2, label='Standard Deviation')

    #ax[1].set_xlim(min(year_range),max(year_range))
    #ax[1].set_xticks(ax[0].get_xticks(),labels=ax[0].get_xticklabels(),rotation=40,ha='right')

    #plt.ylabel('Average number of links per network (95% CI)')
    ax[1].set_ylabel('Num. countries')
    ax[1].title.set_text('Countries trading each food group (median + 95% CI)')
    ax[1].set_xlabel('Year')
    
    # 
    fig,ax= plt.subplots(figsize=(10, 8))
    
    sns.heatmap(year_estimation_links, cmap=sns.color_palette("viridis", as_cmap=True), fmt='g', cbar=True,ax=ax,norm=LogNorm(vmin=1))
    ax.set_facecolor('gray')
    ax.set_title ('Number of transactions per food group')
    plt.show()
    fig.savefig('prova_'+str(flag_save)+'.pdf')


Keep only Items that are considered food Items.

In [3]:
data = pd.read_pickle('../Data/Trade_geo.pkl')
#data=data.loc[data.value>0,:].reset_index()

country_metadata = pd.read_pickle('../Data/Country_info.pkl')
shape_file = pd.read_pickle('../Data/Shapefile_with_positions.pkl')
print('Num unique products, '+ str(len(data.item.unique())))

# Item list prep
#item_list =pd.read_csv('../Data/raw_trade/Trade_ItemCodes_food_products.csv', sep=',', encoding='utf-8').rename(columns={'Item':'item'})

item_list =pd.read_csv('../Data/raw_trade/Trade_ItemCodes_Isfood_foodex.csv', sep=',', encoding='utf-8').rename(columns={'Item':'item'})
foodex_levels =pd.read_excel('../Data/Foodex_raw/Exposure_Hierarchy_revision2.xlsx').loc[:,['Code','IndentedTree']]

data, item_list = Clean_item_names(data, item_list,foodex_levels)

# Filter food items only: 
data = data.loc[data['Is_food']>0,:].drop(columns=['Is_food'])

print('Unique food products in data after filtering, '+ str(len(data.item.unique())))

# Filter items in foodex 
food_item_list = item_list.loc[item_list.item.isin(data.item.unique()),:]

food_item_list

Num unique products, 558
Unique food products in data after filtering, 431


,Item Code,CPC Code,item,Is_food,L1_foodex,Food_group
4,101,'01195,Canary seed,1,A000J,Grains and grain-based products
5,1016,'02123,Goats,1,A01QR,Meat and meat products
6,1017,'21116,"Meat of goat, fresh or chilled",1,A01QR,Meat and meat products
7,1018,'21156,"Edible offal of goat, fresh, chilled or frozen",1,A01QR,Meat and meat products
8,1021,'22254,"Cheese from milk of goats, fresh or processed",1,A02LR,Milk and dairy products
...,...,...,...,...,...,...
560,978,'21155,"Edible offal of sheep, fresh, chilled or frozen",1,A01QR,Meat and meat products
561,979,'21514,"Sheep fat, unrendered",1,A036M,Animal and vegetable fats and oils and primary...
562,982,'02291,Raw milk of sheep,1,A02LR,Milk and dairy products
563,983,'22249.01,Butter and ghee of sheep milk,1,A02LR,Milk and dairy products


## FoodEx aggregation 
Aggregate data in the food groups defined by the FoodEx standard.

In [4]:

# Agregate data in food groups
data = FoodEx_aggregation(data)
data = data.sort_values(by=['Food_group','year'])
# Save data after agregation and filtering
data.reset_index().to_pickle('../Data/Data_food_groups.pkl')


Fruit and vegetable juices and nectars (including concentrates)
Fruit and fruit products
Vegetables and vegetable products
Composite dishes
Meat and meat products
Grains and grain-based products
Milk and dairy products
Sugar and similar, confectionery and water-based sweet desserts
Coffee, cocoa, tea and infusions
Seasoning, sauces and condiments
Eggs and egg products
Animal and vegetable fats and oils and primary derivatives thereof
Water and water-based beverages
Food products for young population
Alcoholic beverages
      Confectionery including chocolate
Starchy roots or tubers and products thereof, sugar plants
Major isolated ingredients, additives, flavours, baking and processing aids
Products for non-standard diets, food imitates and food supplements
Other ingredients


In [5]:
data

,destin_country_ISO,origin_country_ISO,year,unit,value,origin_country,destin_country,L1_foodex,Food_group
27358957,CI,FR,1986,tonnes,1.0,France,Côte d'Ivoire,A04PE,Confectionery including chocolate
27358972,CI,FR,1986,1000 US$,5.0,France,Côte d'Ivoire,A04PE,Confectionery including chocolate
27359023,CI,HK,1986,tonnes,4.0,"China, Hong Kong SAR",Côte d'Ivoire,A04PE,Confectionery including chocolate
27359024,CI,HK,1986,1000 US$,8.0,"China, Hong Kong SAR",Côte d'Ivoire,A04PE,Confectionery including chocolate
27359113,CI,TH,1986,tonnes,12.0,Thailand,Côte d'Ivoire,A04PE,Confectionery including chocolate
...,...,...,...,...,...,...,...,...,...
26128920,NP,US,2021,1000 US$,0.0,United States of America,Nepal,A03DJ,Water and water-based beverages
26128984,NP,AE,2021,tonnes,0.0,United Arab Emirates,Nepal,A03DJ,Water and water-based beverages
26128994,NP,AE,2021,1000 US$,0.0,United Arab Emirates,Nepal,A03DJ,Water and water-based beverages
26129033,NP,CA,2021,tonnes,1.0,Canada,Nepal,A03DJ,Water and water-based beverages


## Check data availability after product aggregation

We see a very skewed distribution on data availability.

In [ ]:
#data_no = data.loc[(data.Food_group != 'Other ingredients') & (data.unit == '1000 US$'),:]

data_c = data.loc[(data.unit == '1000 US$'),:]

avail_countries, avail_trade, year_range = Check_data_available(data_c)

Plot_data_available(avail_countries,avail_trade,year_range,'og')


Other ingredients has almost no data associated. Therefore, we will remove it (for now)

In [ ]:
data_no = data.loc[(data.Food_group != 'Other ingredients') & (data.unit == '1000 US$'),:]

avail_countries_no, avail_trade_no, year_range_no = Check_data_available(data_no)

Plot_data_available(avail_countries_no,avail_trade_no,year_range_no,'no_other')

# Year aggregation
Agregate data to reduce the number of timepoints and get a better resolution in years with low data. 

In [ ]:
Num_parts= 18
data_agr = Year_aggregation(data_no,Num_parts)

# Check data quality
avail_countries_agr, avail_trade_agr, year_range_agr = Check_data_available(data_agr)

Plot_data_available(avail_countries_agr,avail_trade_agr,year_range_agr,'year')


In [ ]:
type(data_agr.year[0])

# OLD CODES

In [ ]:
avail_countries_agr, avail_trade_agr, year_range_agr = Check_data_available(data_agr)

Plot_data_available(avail_countries_agr,avail_trade_agr,year_range_agr)

# Define matrix
agr_range = data_agr.year.unique()
data_agr_g = data_agr.groupby(['item','unit'])

dict_keys=data_agr_g.groups.keys()
year_est_ag= pd.DataFrame(np.zeros([len(dict_keys),len(agr_range)]),columns=agr_range,index=dict_keys)
year_est_ag_l=year_est_ag.copy()

# Fill matrices
for i in list(dict_keys):
    Countries_trading_per_year(data_agr_g.get_group(i),year_est_ag, i)

    Links_per_year(data_agr_g.get_group(i), year_est_ag_l, i)

In [ ]:
# Repeat plots after filtering: 
year_est_bin= 1*(year_est_ag>0)
mean_size = 100*((year_est_bin).sum())/year_est_ag.shape[0]

fig, ax = plt.subplots(1,2,figsize=(10,5))
ax[0].plot(agr_range,mean_size)
ax[0].set_ylim(0,100)
ax[0].set_xticks(ax[0].get_xticks(),labels=ax[0].get_xticklabels(),rotation=40,ha='right')
ax[0].set_ylabel('Percentage of all products with data(%)')
ax[0].title.set_text('Total products with some trade data')

# Mean number of links per year
mean_links = year_est_ag.median(axis=0)

inf_perc = np.quantile(year_est_ag, 0.025,axis=0)
sup_perc = np.quantile(year_est_ag, 0.975,axis=0)
        
std_links = year_est_ag.std()

ax[1].plot(agr_range, mean_links, color='blue', alpha=0.2, label='Standard Deviation')
ax[1].fill_between(agr_range, inf_perc, sup_perc, color='blue', alpha=0.2, label='Standard Deviation')
ax[1].set_xticks(ax[1].get_xticks(),labels=ax[1].get_xticklabels(),rotation=40,ha='right')

ax[1].set_ylabel('Num. countries')
ax[1].title.set_text('Median number of countries per product (95% CI)')
ax[1].set_xlabel('Year intervals')


fig,axes=plt.subplots(2,1,figsize=(100,20))
sns.heatmap(year_estimation.T, cmap='coolwarm',cbar=True,ax=axes[0],xticklabels=True,vmax=200,vmin=0)
sns.heatmap(year_est_ag.T, cmap='coolwarm',cbar=True,ax=axes[1],vmax=200,vmin=0)
#fig.savefig('prova.pdf')
plt.show()

# Drop series without data for more than half of the years(even after aggregation)
If a product has more than half of the years without transactions:

- **Option 1:** Not enough data to study it. *(Remove it)*
- **Option 2:** Super small and localised product that it is not interesting for global trade. *(Remove it)*

In [ ]:
data_agr_g = data_agr.groupby(['item','unit'])
# Fill matrices
zero_links=(year_est_ag_l==0).sum(axis =1)
to_drop= zero_links.loc[zero_links>=len(agr_range)/2,:]

key_to_drop=list(to_drop.keys())
key_to_drop

In [ ]:
# Filter gropus with not enough data:
data_filt= data_agr.copy()
for key in key_to_drop:
    bool_condit= (data_filt.item == key[0]) & (data_filt.unit == key[1])
    #data_agr.drop(data_agr_g2.get_group(key).index, inplace=True)
    data_filt= data_filt.loc[~bool_condit,:]

# Sanity check
print('Before filtering low data:',len(data_agr.loc[:,['item','unit']].drop_duplicates()))
print('Filtered:',len(data_filt.loc[:,['item','unit']].drop_duplicates()))
print('To remove:',len(key_to_drop))
print('Total products with data:',len(data_filt.item.unique()))

In [ ]:
# Repeat data availability after filtering: 

# Define matrix
data_filt_g = data_filt.groupby(['item','unit'])

dict_keys=data_filt_g.groups.keys()
year_est_ag= pd.DataFrame(np.zeros([len(dict_keys),len(agr_range)]),columns=agr_range,index=dict_keys)
year_est_ag_l=year_est_ag.copy()

# Fill matrices
for i in list(dict_keys):
    Countries_trading_per_year(data_filt_g.get_group(i),year_est_ag, i)

    Links_per_year(data_filt_g.get_group(i), year_est_ag_l, i)


# Plot data availability for aggregates:
year_est_bin= 1*(year_est_ag>0)
mean_size = 100*((year_est_bin).sum())/year_est_ag.shape[0]

fig, ax = plt.subplots(1,2,figsize=(10,5))
ax[0].plot(agr_range,mean_size)
ax[0].set_ylim(0,100)
ax[0].set_xticks(ax[0].get_xticks(),labels=ax[0].get_xticklabels(),rotation=40,ha='right')
ax[0].set_ylabel('Percentage of all products with data(%)')
ax[0].title.set_text('Total products with some trade data')

# Mean number of links per year
mean_links = year_est_ag.median(axis=0)

inf_perc = np.quantile(year_est_ag, 0.025,axis=0)
sup_perc = np.quantile(year_est_ag, 0.975,axis=0)
        
std_links = year_est_ag.std()

ax[1].plot(agr_range, mean_links, color='blue', alpha=0.2, label='Standard Deviation')
ax[1].fill_between(agr_range, inf_perc, sup_perc, color='blue', alpha=0.2, label='Standard Deviation')
ax[1].set_xticks(ax[1].get_xticks(),labels=ax[1].get_xticklabels(),rotation=40,ha='right')

#plt.ylabel('Average number of links per network (95% CI)')
ax[1].set_ylabel('Num. countries')
ax[1].title.set_text('Median number of countries per product (95% CI)')
ax[1].set_xlabel('Year intervals')

In [ ]:
# Compare filtered samples from original estimation: 
filt_compare = year_estimation.loc[year_est_ag.index,:]

fig,axes=plt.subplots(2,1,figsize=(100,20))
sns.heatmap(filt_compare.T, cmap='coolwarm',cbar=True,ax=axes[0],xticklabels=False, vmax=200,vmin=0)
sns.heatmap(year_est_ag.T, cmap='coolwarm',cbar=True,ax=axes[1],vmax=200,vmin=0)
#fig.savefig('prova.pdf')
plt.show()

# Product aggregation: 
Use FoodEx Level 1 classification to aggregate products into food categories (much simpler analysis).

In [ ]:
# Save data after agregation and filtering
data_filt.reset_index().to_pickle('../Data/Data_foodex_groups_'+str(Num_parts)+'.pkl')

In [ ]:
data_filt